In [1]:
from binance import Client
from datetime import datetime, timedelta
client = Client(api_key="jM9zEeTTzT7R7QGlNrh5tUpOc8l71hOxu7CewCL4M0NvbBp3ilgmEwMOOjJfCRVB",api_secret="iZxFmlz5bvROsHQPZTP5hlJtsxYjOu8wocOlLWzIDdhMzEQu7KyyHB2DseOYQ66s",testnet=True)

import os
from google.cloud.sql.connector import Connector
from sqlalchemy import create_engine, text
import pg8000.native
import pandas as pd

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/tinma/OneDrive/Escritorio/application_default_credentials.json'

connector = Connector()

def getconn() -> pg8000.native.Connection:
    conn: pg8000.native.Connection = connector.connect(
        "semiotic-pact-388619:us-central1:tradingbot",
        "pg8000",
        user="postgres",
        password="GuruSat.3",
        db="criptos"
    )
    return conn

connection = create_engine('postgresql+pg8000://',creator=getconn)

def MACD(df):
    df['EMA12'] = df.Precio.ewm(span=12).mean()
    df['EMA26'] = df.Precio.ewm(span=26).mean()
    df['MACD'] = df.EMA12 - df.EMA26
    df['Signal'] = df.MACD.ewm(span=9).mean()
    # print('Indicadores agregados')

def señal_MACD(df):
    buy, sell = [],[]
    for i in range(7,len(df)):
        if df.MACD.iloc[i] > df.Signal.iloc[i] and df.MACD.iloc[i-1] < df.Signal.iloc[i-1]:
            buy.append(i)
        elif df.MACD.iloc[i] < df.Signal.iloc[i] and df.MACD.iloc[i-1] > df.Signal.iloc[i-1]:
            sell.append(i)
    return buy, sell

In [2]:
path = 'C:/Users/tinma/OneDrive/Escritorio/tradingbot/trade_logs/testnet/MACD/'

balances_p = []
balances_c = []

PAIR = 'BNBUSDT'
pair = 'bnb'
LOCALIDAD_SEÑAL_ENTRADA = 10
LOCALIDAD_SEÑAL_SALIDA = 10

open_order = False

while True:

    day = datetime.utcnow() - timedelta(seconds=1000)

    with connection.connect() as conn:
        df =conn.execute(text(""" SELECT * FROM "{PAIR}" WHERE "Tiempo" > '{day}' """.format(day=day,PAIR=PAIR))).mappings().all()
    df = pd.DataFrame(df)[['Tiempo', 'Precio']].set_index('Tiempo')
    df = df.sort_values(by='Tiempo')
    MACD(df)
    buy, sell = señal_MACD(df)

    buyprices = df.Precio.iloc[buy]
    sellprices = df.Precio.iloc[sell]






    if open_order == False:

        DIFF_SEÑAL_ENTRADA = (datetime.utcnow() - buyprices.index[-1]).total_seconds()

        if DIFF_SEÑAL_ENTRADA < LOCALIDAD_SEÑAL_ENTRADA:
            balance_inicial = float(client.get_account()['balances'][6]['free'])
            open_order = True
            buy_order = client.create_order(
                        symbol=PAIR,
                        side=Client.SIDE_BUY,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.1
                    )
            tiempo_entrada = datetime.utcnow()
            precio_entrada = float(buy_order['fills'][0]['price'])
            buy_id = buy_order['orderId']

            # tiempos_entrada.append(tiempo_entrada)
            # precios_entrada.append(precio_entrada)
            # buy_ids.append(buy_id)
            print(f'Se realizó una compra al precio: {precio_entrada}, en el momento: {tiempo_entrada}. Con un balance inicial de: {balance_inicial}')

    if open_order == True:

        if df.Precio[-1] - precio_entrada < -3:
            open_order = False
            sell_order = client.create_order(
                        symbol=PAIR,
                        side=Client.SIDE_SELL,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.1
                    )
            tiempo_salida = datetime.utcnow()
            precio_salida = float(sell_order['fills'][0]['price'])
            sell_id = sell_order['orderId']
            balance_final = float(client.get_account()['balances'][6]['free'])
            balance_trade_p = (precio_salida - precio_entrada)
            balance_trade_c = balance_final - balance_inicial
            balances_p.append(balance_trade_p*0.1)
            balances_c.append(balance_trade_c)
            print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}.')

            if balance_trade_c>0:
                print('Según precios de entrada y salida, la ganancia fue de:', (balance_trade_p)*0.1, 'dólares. Umbral de pérdida')
                print('Según el balance inicial y final de la cuenta después del trade, la ganancia fue de:', (balance_trade_c), 'dólares')
                trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance_trade_c)]],
                                    columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Ganancia'])
                if os.path.exists(path+f'trades_ganados_{pair}.csv'):
                    trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
                                    mode='a',
                                    index=False,
                                    header=False
                                    )
                else:
                    trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
                                    index=False,
                                    header=True,
                                    )
            elif balance_trade_c==0:
                print('No hubo pérdidas ni ganancias del trade')
            else:
                print('Según precios de entrada y salida, la pérdida fue de:', (balance_trade_p)*0.1, 'dólares. Umbral de pérdida')
                print('Según el balance inicial y final de la cuenta después del trade, la pérdida fue de:', (balance_trade_c), 'dólares')
                trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance_trade_c)]],
                                    columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Perdida'])
                if os.path.exists(path+f'trades_perdidos_{pair}.csv'):
                    trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
                                    mode='a',
                                    index=False,
                                    header=False
                                    )
                else:
                    trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
                                    index=False,
                                    header=True,
                                    )

    if open_order == True:

        DIFF_SEÑAL_SALIDA = (datetime.utcnow() - sellprices.index[-1]).total_seconds()

        if DIFF_SEÑAL_SALIDA < LOCALIDAD_SEÑAL_SALIDA and sellprices.index[-1] > buyprices.index[-1]:
            open_order = False
            sell_order = client.create_order(
                        symbol=PAIR,
                        side=Client.SIDE_SELL,
                        type=Client.ORDER_TYPE_MARKET,
                        quantity=0.1
                    )
            tiempo_salida = datetime.utcnow()
            precio_salida = float(sell_order['fills'][0]['price'])
            sell_id = sell_order['orderId']
            balance_final = float(client.get_account()['balances'][6]['free'])
            balance_trade_p = (precio_salida - precio_entrada)
            balance_trade_c = balance_final - balance_inicial
            balances_p.append(balance_trade_p*0.1)
            balances_c.append(balance_trade_c)
            print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}.')

            if balance_trade_c>0:
                print('Según precios de entrada y salida, la ganancia fue de:', (balance_trade_p)*0.1, 'dólares. Señal')
                print('Según el balance inicial y final de la cuenta después del trade, la ganancia fue de:', (balance_trade_c), 'dólares')
                trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance_trade_c)]],
                                    columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Ganancia'])
                if os.path.exists(path+f'trades_ganados_{pair}.csv'):
                    trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
                                    mode='a',
                                    index=False,
                                    header=False
                                    )
                else:
                    trade_log.to_csv(path+f'trades_ganados_{pair}.csv',
                                    index=False,
                                    header=True,
                                    )
            elif balance_trade_c==0:
                print('No hubo pérdidas ni ganancias del trade')
            else:
                print('Según precios de entrada y salida, la pérdida fue de:', (balance_trade_p)*0.1, 'dólares. Señal')
                print('Según el balance inicial y final de la cuenta después del trade, la pérdida fue de:', (balance_trade_c), 'dólares')
                trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance_trade_c)]],
                                    columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Perdida'])
                if os.path.exists(path+f'trades_perdidos_{pair}.csv'):
                    trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
                                    mode='a',
                                    index=False,
                                    header=False
                                    )
                else:
                    trade_log.to_csv(path+f'trades_perdidos_{pair}.csv',
                                    index=False,
                                    header=True,
                                    )

Se realizó una compra al precio: 244.6, en el momento: 2023-07-02 02:53:53.462088. Con un balance inicial de: 6712.64544451
Se realizó una venta al precio: 244.5, en el momento: 2023-07-02 02:54:44.397719.
Según precios de entrada y salida, la pérdida fue de: -0.009999999999999433 dólares. Señal
Según el balance inicial y final de la cuenta después del trade, la pérdida fue de: -0.010000000000218279 dólares
Se realizó una compra al precio: 244.6, en el momento: 2023-07-02 02:54:52.404203. Con un balance inicial de: 6712.63544451
Se realizó una venta al precio: 244.49, en el momento: 2023-07-02 02:55:06.265852.
Según precios de entrada y salida, la pérdida fue de: -0.010999999999998523 dólares. Señal
Según el balance inicial y final de la cuenta después del trade, la pérdida fue de: -0.01099999999951251 dólares
Se realizó una compra al precio: 244.5, en el momento: 2023-07-02 02:55:43.558463. Con un balance inicial de: 6712.62444451
Se realizó una venta al precio: 244.51, en el momento:

KeyboardInterrupt: 

In [4]:
sum(balances_c)

-1.6977999999999156

In [5]:
open_order

False